# Answering Epistemic Questions

In this case study, we use GLIF to answer yes/no question that require handling the knowledge of different people.
For example, given the input

    John knows that Mary or Eve knows that Ping has a dog.
    Mary doesn't know if Ping has a dog.
    Does Eve know if Ping has a dog?
    
the system should reply *"Yes"*.

In [1]:
archive tmpGLIF/examples epistemic

Successfully changed archive

## Grammar

First, we have to be able to parse the sentences.
While the grammar (or at least, the abstract syntax) is fairly easy,
we would have to take care of the endings. Consider, for example, the following sentences:
* *John has a dog*
* *John doesn't have a dog*
* *John and Mary have a dog*

GF has mechanisms to take care of the verb forms (*has* vs *doesn't have* vs *have*),
but rather than doing this ourselves, we will use the [RGL (Resource Grammar Library)](https://www.grammaticalframework.org/lib/doc/synopsis/index.html) that comes with GF and implements morphology and basic syntax of lots of languages.

As a consequence, we inherit the maybe slightly un-intuitive but useful distinction between a full sentence and a clause (`Cl`).
A `Cl` already has a noun phrase and a verb phrase, but it still has to be turned into a sentence.
For example, a `Cl` might have *John* and *to have a dog*, and we turn it into different sentences:
* *John has a dog*
* *John doesn't have a dog*
* *does John have a dog?*

With RGL, we could also use other tenses, but that's not relevant for us.

In [2]:
abstract Epistemic = {
    cat
        S;          -- a full sentence (statement or question)
        Cl;         -- a clause (almost a sentence, but can still be turned into a question/statement)
        Agent;      -- "John"
        NP;         -- "John and Mary"
        VP;         -- "to have a dog" or "to love John"
    
    fun
        state : Cl -> S;            -- "John has a dog"
        stateNot : Cl -> S;         -- "John doesn't have a dog"
        ask : Cl -> S;              -- "Does John have a dog?"

        simpleCl : NP -> VP -> Cl;  -- "John has a dog"
        knowCl : NP -> Cl -> Cl;    -- "Mary knows that John has a dog"
        knowNotCl : NP -> Cl -> Cl; -- "Mary knows that John doesn't have a dog"
        knowWhetherCl : NP -> Cl -> Cl;  -- "Mary knows whether John has a dog"
        
        npify : Agent -> NP;        -- "John"
        andNP : NP -> NP -> NP;     -- "John and Mary"
        orNP : NP -> NP -> NP;      -- "John or Mary"
        
        john, mary, eve, ping : Agent;
        have_cat, have_dog : VP;
        love, hate : NP -> VP;
}

Successfully imported Epistemic.gf

In the **concrete syntax**, we map the syntactic categories from our abstract syntax to types from the RGL.
Similarly, we provide linearizations for the rules using RGL operations.

You will see that the input/output does not exactly match our expectations.
Concretely, there will be a space before punctuation and questions starting with *do*/*does* start with a lowercase letter.
That's because GF works with tokens, which are, by default, space-separated.
We could fix this with relatively simple pre-processing.

In [3]:
concrete EpistemicEng of Epistemic = open SyntaxEng, ParadigmsEng, ParamX in {
    lincat
        S = Text;
        Cl = Cl;
        Agent = PN;
        NP = NP;
        VP = VP;

    lin
        state cl = mkText (mkS positivePol cl);
        stateNot cl = mkText (mkS negativePol cl);
        ask cl = mkText (mkQS cl);
        
        simpleCl p v = mkCl p v;
        knowCl p cl = mkCl p (mkVS (mkV "know")) (mkS positivePol cl);
        knowNotCl p cl = mkCl p (mkVS (mkV "know")) (mkS negativePol cl);
        knowWhetherCl np cl = mkCl np (mkVP (mkVQ (mkV "know")) (mkQS (mkQCl cl)));

        npify a = mkNP a;
        andNP a b = mkNP and_Conj a b;
        orNP a b = mkNP or_Conj a b;

        john = mkPN "John";
        mary = mkPN "Mary";
        eve = mkPN "Eve";
        ping = mkPN "Ping";
        have_cat = mkVP have_V2 (mkNP aSg_Det (mkN "cat"));
        have_dog = mkVP have_V2 (mkNP aSg_Det (mkN "dog"));
        love p = mkVP (mkV2 (mkV "love")) p;
        hate p = mkVP (mkV2 (mkV "hate")) p;
}

Successfully imported EpistemicEng.gf

In [4]:
-- some random sentences
gr -number=5 -depth=5 | l

Mary and Ping and Ping or Eve don't know that Eve doesn't know that Ping has a dog . Eve or Eve and Eve don't know that Eve hates Mary . do Eve and Eve have a dog ? do Ping and John and John and Eve know that Mary or John has a dog ? Eve hates Mary .

In [5]:
-- and some visualizations
parse "John loves Mary ." "does John love Mary ?" "John and Mary love Ping ." "John doesn't love Mary ." "John knows that Mary has a dog ." | vp -showfun

Dropdown(layout=Layout(width='max-content'), options=('0.0. state (simpleCl (npify john) (love (npify mary)))'…

Image(value=b'<?xml version="1.0" encoding="UTF-8" sta...', format='svg+xml')

## Logic

We will represent the logic in the multi-modal logic S5n.
The box operator is used to state that someone knows something.
As we have a separate operator for every person, we will use the notation
⟦A⟧φ to indicate that A knows that φ is true.

For example, *John knows that Mary loves Eve* would be represented as

    ⟦john⟧(love mary eve)
    

In [6]:
theory proplog : ur:?LF =
    proposition : type ❘ # o ❙
    neg : o ⟶ o ❘ # ¬ 1 prec 80 ❙
    and : o ⟶ o ⟶ o ❘ # 1 ∧ 2 prec 60 ❙
    or : o ⟶ o ⟶ o ❘ # 1 ∨ 2 prec 50 ❙
    imp : o ⟶ o ⟶ o ❘ # 1 ⇒ 2 prec 40 ❙
❚ 

Successfully imported proplog.mmt

In [7]:
theory s5n : ur:?LF =
    include ?proplog ❙
    agent : type ❘ # ι ❙
    box : ι ⟶ o ⟶ o ❘ # ⟦ 1 ⟧ 2 prec 10 ❙
    dia : ι ⟶ o ⟶ o ❘ # ⟪ 1 ⟫ 2 prec 10 ❙
❚

Successfully imported s5n.mmt

In [8]:
theory utterances : ur:?LF =
    include ?proplog ❙
    // After the semantics construction, we will still want to distinguish statements and questions ❙
    utterance : type ❙
    statement : o ⟶ utterance ❘ # ! 1 ❙
    question : o ⟶ utterance ❘ # ? 1 ❙
❚

Successfully imported utterances.mmt

In [9]:
theory epistemicDDT : ?proplog =
    // The "discourse domain theory" ❙
    // Sentences will be translated to expressions in this theory ❙
    include ?utterances ❙
    include ?s5n ❙
    john : ι ❙
    mary : ι ❙
    eve : ι ❙
    ping : ι ❙
    havedog : ι ⟶ o ❙
    havecat : ι ⟶ o ❙
    love : ι ⟶ ι ⟶ o ❙
    hate : ι ⟶ ι ⟶ o ❙
❚

Successfully imported epistemicDDT.mmt

In the **semantics construction view** we map all symbols from the abstract syntax to expressions in the target theory `epistemicDDT`.

For noun phrases (`NP`), we use **type raising**.
While intuitively, the type of *John* should be ι, that doesn't work anymore for complex noun phrases like *John and Mary*.
Instead, we use the well-established trick and use type (ι⟶o)⟶o.
*John and Mary* would then be represented as \[p\] p john ∧ p mary.
Whatever we want to say about John and Mary, we can then say by choosing an appropriate predicate p.
For example, *John and Mary have a dog* would be translated to (\[p\] p john ∧ p mary) havedog.

Below, we will have a few examples that might help illustrate this.

In [10]:
view EpistemicSemantics : http://mathhub.info/tmpGLIF/examples/epistemic/Epistemic.gf?Epistemic -> ?epistemicDDT =
    S = utterance ❙
    Cl = o ❙
    Agent = ι ❙
    NP = (ι ⟶ o) ⟶ o ❙
    VP = ι ⟶ o ❙

    state = [cl] statement cl ❙
    stateNot = [cl] statement (¬(cl)) ❙
    ask = [cl] question cl ❙
    simpleCl = [np,vp] np vp ❙
    knowCl = [np,s] np ([x] ⟦ x ⟧ s) ❙
    knowNotCl = [np,s] np ([x] ⟦ x ⟧ ¬(s)) ❙
    knowWhetherCl = [np,cl] (np ([x] ⟦ x ⟧ cl)) ∨ (np ([x] ⟦ x ⟧ ¬ (cl))) ❙

    npify = [x] [p] p x ❙
    andNP = [a,b] [p] (a p) ∧ (b p) ❙
    orNP = [a,b] [p] (a p) ∨ (b p) ❙
    
    john = john ❙
    mary = mary ❙
    eve = eve ❙
    ping = ping ❙
    have_cat = havecat ❙
    have_dog = havedog ❙
    love = [np] [x] np (love x) ❙
    hate = [np] [x] np (hate x) ❙
❚

Successfully imported EpistemicSemantics.mmt

In [11]:
p "John has a dog ."
p "John has a dog ." | construct -no-simplify
p "John has a dog ." | construct

state (simpleCl (npify john) have_dog)

([cl]!cl) (([np,vp]np vp) (([x][p]p x) john) havedog)

!(havedog john)

In [12]:
p "does John have a dog ?" | construct

?(havedog john)

In [13]:
-- two ways to parse this ⟶ two logical expressions
p "John and Mary and Ping have a dog ." | construct

!(havedog john)∧((havedog mary)∧(havedog ping)) !(havedog john)∧(havedog mary)∧(havedog ping)

In [14]:
p -cat=NP "John and Mary" | construct

[p](p john)∧(p mary)

In [15]:
p "John knows that Mary loves John ." | construct

!⟦john⟧(love mary john)

In [16]:
p "John doesn't know if Mary and Ping have a dog ." | construct

!¬((⟦john⟧(havedog mary)∧(havedog ping))∨⟦john⟧¬((havedog mary)∧(havedog ping)))

In [17]:
p "John knows that Mary loves John and Eve ." | construct

!⟦john⟧(love mary john)∧(love mary eve)

In [18]:
-- We can also do sequences of sentences
p "John has a dog ." "Mary has a dog ." "does Eve have a dog ?" | construct

!(havedog john) !(havedog mary) ?(havedog eve)

## Inference

For the inference, we need an S5n prover.
In this case, we adapted a Prolog prover from *de Boer, M. S. (2006). Praktische Bewĳzen in Public Announcement Logica*.
Type-checking is optional in ELPI and in this case we do not use it because the Prolog was not designed with that in mind.


In [19]:
elpi-notc: util

% the original prover was implemented in normal Prolog, so we need some helper predicates.

select A [A] [] :- !.
select A [A|L] L.
select A [X|T] [X|R] :- select A T R.

freeze X f :- var X, !, declare_constraint (f X) [X].
freeze X f :- f X.

member X [X|_] :- !.
member X [_|L] :- member X L.

Successfully imported util.elpi util.elpi is the new default file for ELPI commands

In [20]:
elpi-notc: s5npal

accumulate util.

% Adapted from: de Boer, M. S. (2006). Praktische Bewĳzen in Public Announcement Logica.

% This is probably not very understandable – you may just accept it as a blackbox.

p F :- prove [l [] (neg F)] [], !.

prove [LFml|Fs] Branch :- 
    (ruleOne LFml _tpe NLF, !, prove [NLF|Fs] Branch);
    (ruleTwo LFml conj A B, !, prove [A,B|Fs] Branch).
prove [l L Fml|Fs] Branch :-
    (Fml = neg Neg; Neg = neg Fml),
    (memberunify (l L Neg) Branch; prove Fs [l L Fml | Branch]), !.
prove [] Branch :-
    select (l Label Fml) Branch RestB,
    ruleTwo (l _ Fml) disj (l _ A) (l _ B),
    expand Label Inst M _,
    (Label = l _ L; Label = L), !,
    prove [l M A] [l (l Inst L) Fml|RestB],
    prove [l M B] [l (l Inst L) Fml|RestB].

expand (l [[X|Xs]|Xss] [star I|Ls]) [[X|Xs]|Yss] [l X I|Ms] G :-
    var X, !,
    expand (l Xss Ls) Yss Ms G.
expand (l [Xs|Xss] [star I|Ls]) [[X|Xs]|Yss] [l X I|Ms] _ :- !,
    freeze X (x \ not (member X Xs)),
    expand (l Xss Ls) Yss Ms true.
expand (l Xss [C|Ls]) Yss [C|Ms] G :- !,
    expand (l Xss Ls) Yss Ms G.
expand (l [] []) [] [] G :-
    not (var G).
expand [star I|Ls] [[X]|Xss] [l X I|Ms] _ :- !,
    expand Ls Xss Ms _.
expand [L|Ls] Xss [L|Ms] _ :- !,
    expand Ls Xss Ms _.
expand [] [] [] _.

memberunify (l L A) Ls :-
    member (l K A) Ls,
    red O L [],
    red O K [], !.

red O [l _ I|L] [l X I|R] :- !,
    red O L [l X I|R].
red O L [l O _|R] :-
    red O L R.
red O L [star I|R] :- !,
    red O L [l _ I|R].
red O [X|L] R :- !,
    red O L [X|R].
red O [] O.

ruleTwo (l L (and A B)) conj (l L A) (l L B).
ruleTwo (l L (neg (imp A B))) conj (l L A) (l L (neg B)).
ruleTwo (l L (neg (or A B))) conj (l L (neg A)) (l L (neg B)).
ruleTwo (l L (or A B)) disj (l L A) (l L B).
ruleTwo (l L (imp A B)) disj (l L (neg A)) (l L B).
ruleTwo (l L (neg (and A B))) disj (l L (neg A)) (l L (neg B)).
ruleTwo (l L (eq A B)) disj (l L (and A B)) (l L (and (neg A) (neg B))).
ruleTwo (l L (neg (eq A B))) disj (l L (and A (neg B))) (l L (and (neg A) B)).

ruleOne (l L (neg (bra F))) doub (l L (neg F)).
ruleOne (l L (bra F)) doub (l L F).
ruleOne (l L (neg (neg F))) doub (l L F).

ruleOne (l [l _ I|L] (box I F)) know (l [star I|L] F).
ruleOne (l [l _ I|L] (neg (dia I F))) know (l [star I|L] (neg F)).
ruleOne (l [star I|L] (box I F)) know (l [star I|L] F).
ruleOne (l [star I|L] (neg (dia I F))) know (l [star I|L] (neg F)).
ruleOne (l L (box I F)) know (l [star I|L] F).
ruleOne (l L (neg (dia I F))) know (l [star I|L] (neg F)).

ruleOne (l [l _ I|L] (dia I F)) poss (l [l F I|L] F).
ruleOne (l [l _ I|L] (neg (bos I F))) poss (l [l F I|L] (neg F)).
ruleOne (l [star I|L] (dia I F)) poss (l [l F I|L] F).
ruleOne (l [star I|L] (neg (bos I F))) poss (l [l (neg F) I|L] (neg F)).
ruleOne (l L (dia I F)) poss (l [l F I|L] F).
ruleOne (l L (neg (box I F))) poss (l [l (neg F) I|L] (neg F)).

Successfully imported s5npal.elpi s5npal.elpi is the new default file for ELPI commands

In [21]:
-- generate ELPI signatures for discourse domain theory and the grammar
elpigen -mode=types epistemicDDT

Successfully created epistemicDDT.elpi

## Question answering

If we have premises (statements) P1, ..., Pn and a question Q, then we want to reply
* **Yes** if P1 ∧ ... ∧ Pn ⊢ Q (and not P1 ∧ ... ∧ Pn ⊢ ⊥)
* **No** if P1 ∧ ... ∧ Pn ⊢ ¬Q
* **That doesn't make sense** if P1 ∧ ... ∧ Pn ⊢ ⊥ (i.e. the premises are inconsistent)
* **Unknown** otherwise

Below, we have some code that uses the prover from above to answer questions.
GLIF will call the `apply` predicate with a list of items.
An **item** is in this case a logical expression, but it also carries the original sentence, which lets us print the questions we are answering in English.

In [22]:
elpi-notc: answerer

accumulate s5npal.
accumulate epistemicDDT.

% Question answering as explained above.
% We use implication (`imp`) as we have A ⇒ B is valid iff A ⊢ B.
answer Premises _Conclusion :-
    p (imp Premises falsum), !, print "  ⟶ That doesn't make sense!".
answer Premises Conclusion :-
    p (imp Premises Conclusion), !, print "  ⟶ Yes!".
answer Premises Conclusion :-
    p (imp Premises (neg Conclusion)), !, print "  ⟶ No!".
answer _Premises _Conclusion :-
    print "  ⟶ Unknown...".

% A sequence of sentences might have multiple questions.
% `process_items` accumulates statements in a conjunction (`Premises`).
% Whenever a question occurs, it answers it with the premises accumulated so far.

% No sentences left -> Done
process_items Premises [].

% A question -> answer it
process_items Premises [Head|Tail] :-
    glif.getLog Head (question X), !,   % process questions
    glif.getStr Head S, print S,        % print the question for reference
    answer Premises X,
    process_items Premises Tail.        % answer remaining questions

% A statement that leads to a contradiction -> point out the contradiction and stop
process_items Premises [Head|Tail] :-
    glif.getLog Head (statement X),
    p (imp (and Premises X) falsum), !,
    glif.getStr Head S, print S,        % print the statement for reference
    print "... That doesn't make any sense... I'll stop listening now!".

% A statement -> add it to the premises
process_items Premises [Head|Tail] :-
    glif.getLog Head (statement X),
    glif.getStr Head S, print S,        % print the statement for reference
    process_items (and Premises X) Tail.% add to premises and answer remaining questions

% the apply command will be called by GLIF.
apply Items :-
    process_items truth Items.

Successfully imported answerer.elpi answerer.elpi is the new default file for ELPI commands

## Let's try it out

In [23]:
p "John loves Mary ." "does John love Mary ?" | construct | apply -all -notc

John loves Mary . does John love Mary ?   ⟶ Yes!

In [24]:
p "John doesn't love Mary ." "does John love Mary ?" | construct | apply -all -notc

John doesn't love Mary . does John love Mary ?   ⟶ No!

In [25]:
p "John knows that Mary loves Eve ." "does Mary love Eve ?" | construct | apply -all -notc

John knows that Mary loves Eve . does Mary love Eve ?   ⟶ Yes!

In [26]:
p "John loves Eve ." "does John love Mary ?" | construct | apply -all -notc

John loves Eve . does John love Mary ?   ⟶ Unknown...

In [27]:
p "John loves Mary ." "does John love Mary ?" | construct | apply -all -notc
p "John loves Mary ." "John doesn't love Mary ." "does John love Mary ?" | construct | apply -all -notc
p "John loves Eve ." "does John love Mary ?" | construct | apply -all -notc

John loves Mary . does John love Mary ?   ⟶ Yes!

John loves Mary . John doesn't love Mary . ... That doesn't make any sense... I'll stop listening now!

John loves Eve . does John love Mary ?   ⟶ Unknown...

In [28]:
-- The example from the introduction
p "John knows that Mary or Eve knows that Ping has a dog ." "Mary doesn't know if Ping has a dog ." "does Eve know if Ping has a dog ?" | construct
p "John knows that Mary or Eve knows that Ping has a dog ." "Mary doesn't know if Ping has a dog ." "does Eve know if Ping has a dog ?" | construct | apply -all -notc

!⟦john⟧(⟦mary⟧(havedog ping))∨⟦eve⟧(havedog ping) !¬((⟦mary⟧(havedog ping))∨⟦mary⟧¬(havedog ping)) ?(⟦eve⟧(havedog ping))∨⟦eve⟧¬(havedog ping)

John knows that Mary or Eve knows that Ping has a dog . Mary doesn't know if Ping has a dog . does Eve know if Ping has a dog ?   ⟶ Yes!